# Import libraries

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

## Import DataSet

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
#we don't need first 3 columns as they are irrelevant for classification
x = dataset.iloc[:,3:-1].values
#only last column is for classification
y = dataset.iloc[:,-1].values

In [3]:
print(x)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


##**Encoding Categorical Data**

Label Encoding the Gender Column

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:,2] = le.fit_transform(x[:,2])

In [5]:
print(x)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One hot encoding for "country" column

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough') #applying one hot encoding in 1st column
x = np.array(ct.fit_transform(x))

In [7]:
print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


##Split DataSet in Training and Test dataset

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

##Feature Scaling


In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

## **Building ANN**

In [10]:
ann = tf.keras.models.Sequential()

Adding Input Layer and First Hidden Layer

In [11]:
ann.add(tf.keras.layers.Dense(6, activation = 'relu')) #this value of 6 we got from experiments

Add Second Hidden Layer

In [12]:
ann.add(tf.keras.layers.Dense(4, activation = 'relu')) #this value of 4 we got from experiments

Add Output Layer

In [13]:
ann.add(tf.keras.layers.Dense(1, activation = 'sigmoid')) #this value of 1, because this is single layer classification. either we get 0 or 1
#sigmoid gives better probability for binary outcomes

## **Training the ANN** 

Compiling ANN

In [14]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

Train the ANN

In [15]:
ann.fit(x_train,y_train,batch_size=32,epochs=100)

Epoch 1/100
250/250 [==============================] - 2s 2ms/step - loss: 0.5176 - accuracy: 0.7960
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4674 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4478 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4383 - accuracy: 0.7960
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4328 - accuracy: 0.7960
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4293 - accuracy: 0.7960
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4267 - accuracy: 0.7960
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4241 - accuracy: 0.7956
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4220 - accuracy: 0.8054
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4184 - accura

##Making the predictions and evaluating the model

Use our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: $ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: $ 50000

So, should we say goodbye to that customer?

In [16]:
probability = ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]]))

1/1 [==============================] - 0s 237ms/step


In [17]:
print(probability)

[[0.02496733]]


##Predicting the Test set results

In [19]:
y_pred = ann.predict(x_test)
y_pred = y_pred > 0.5
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


##Confusion Matrix

In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1504   91]
 [ 191  214]]


0.859